In [1]:
import os
import math
import numpy as np
from PIL import Image
from u_net import get_unet_128
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
batch_size = 3
input_shape = (320,240,3)
x_dir = "../simple_model/data/train/images/"
y_dir = "../simple_model/data/train/masks/"

In [3]:
x_files = os.listdir(x_dir)
y_files = os.listdir(y_dir)
all_files = list(zip(x_files, y_files))

In [51]:
def grab_batch(file_list):

    while True:
        x, y = [], []

        for i in range(len(file_list)):
            x.append(np.array(Image.open(x_dir + file_list[i][0])))
            y.append(np.array(Image.open(y_dir + file_list[i][1]).convert('L')))

            if ((len(x) == batch_size) or (i == len(file_list) - 1)):
                x = np.array(x)
                y = np.array(y)
                yield(x, y)
                x, y = [], []

In [93]:
train_files, test_files = train_test_split(all_files, test_size = 0.2)

model = get_unet_128(input_shape)

model.fit_generator(grab_batch(train_files),
                    steps_per_epoch = math.ceil(len(train_files) / batch_size),
                    validation_data = grab_batch(test_files),
                    validation_steps = math.ceil(len(test_files) / batch_size),
                    epochs = 2)

Epoch 1/2
5/5 [==============================] - 2s 363ms/step - loss: -406.6928 - dice_loss: 1.9111 - val_loss: -568.6899 - val_dice_loss: 1.9231
Epoch 2/2
5/5 [==============================] - 0s 77ms/step - loss: -430.7607 - dice_loss: 1.9527 - val_loss: -547.0114 - val_dice_loss: 1.9011


In [106]:
kf = KFold(n_splits = 5)
all_files = np.array(all_files)

for train_index, test_index in kf.split(all_files):

    train_files, test_files = all_files[train_index], all_files[test_index]

    model = get_unet_128(input_shape)
    
    model.fit_generator(grab_batch(train_files),
                        steps_per_epoch = math.ceil(len(train_files) / batch_size),
                        validation_data = grab_batch(test_files),
                        validation_steps = math.ceil(len(test_files) / batch_size),
                        epochs = 2)

Epoch 1/2
5/5 [==============================] - 7s 1s/step - loss: -103.3340 - dice_loss: 1.3580 - val_loss: -854.9723 - val_dice_loss: 1.9652
Epoch 2/2
5/5 [==============================] - 0s 78ms/step - loss: -252.6804 - dice_loss: 1.8182 - val_loss: -11.3173 - val_dice_loss: 1.0092
Epoch 1/2
5/5 [==============================] - 7s 1s/step - loss: -144.7828 - dice_loss: 1.5487 - val_loss: -703.2266 - val_dice_loss: 1.9660
Epoch 2/2
5/5 [==============================] - 0s 78ms/step - loss: -293.3208 - dice_loss: 1.8563 - val_loss: -710.2056 - val_dice_loss: 1.9786
Epoch 1/2
5/5 [==============================] - 8s 2s/step - loss: -158.6882 - dice_loss: 1.6069 - val_loss: -699.9170 - val_dice_loss: 1.9680
Epoch 2/2
5/5 [==============================] - 0s 79ms/step - loss: -309.0218 - dice_loss: 1.8844 - val_loss: -695.7798 - val_dice_loss: 1.9753
Epoch 1/2
5/5 [==============================] - 8s 2s/step - loss: -183.7210 - dice_loss: 1.5911 - val_loss: -596.2846 - val_dice_